In [ ]:
import numpy as np
from network import Network
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits import mplot3d
import metrics
import math

plt.rcParams.update({'font.size': 20})
plt.rc('figure', figsize=(20, 10))
plt.rc('figure', facecolor=(1,1,1,1))


## Ejercicio 3

### 3.a

In [ ]:
lim = 1
size = 100
x = np.linspace(-lim, lim, size)
y = np.linspace(-lim, lim, size)
sample = [[i, j] for i in x for j in y]


def plot(network):
    img = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            img[i][j] = network.feedforward([x[i], y[j]])[0]
    plt.xlabel("E1")
    plt.ylabel("E2")
    plt.pcolormesh(x, y, img, vmin=-1, vmax=1,
                   shading="gouraud", cmap="coolwarm")
    plt.colorbar()


In [ ]:
dataset = [
    [[-1, -1], [-1]],
    [[-1, 1], [1]],
    [[1, -1], [1]],
    [[1, 1], [-1]]
]

network = Network(structure=(
    [2, 2, 1]), activation='tanh', seed=17, args={'b': 0.8})
errors = network.train(dataset, learning_rate=0.1,
                       target_error=0.01, momentum=0.9)
print("Epochs: ", len(errors))
print("Error: {:.2e}".format(network.error(dataset)))
plot(network)
plt.show()

#### Input variation

##### Averages

In [ ]:
def run(dataset,structure, epochs = math.inf, learning_rate = 0.1, target_error = 0, momentum=0, activation='tanh', seed=17, b=1):

    network = Network(structure=structure, activation=activation, seed=seed, args = {"b":b})
    errors = network.train(dataset, epochs=epochs, learning_rate=learning_rate,
                           target_error=target_error, momentum=momentum)

    return len(errors)

seeds = [i+1 for i in range(5)]

In [ ]:

momentums = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
learning_rates = [0.1, 0.2, 0.3]


plt.figure("avg epochs vs momentums")

for lr in learning_rates:
    avg = []
    errors = []

    for m in momentums:
        epochs = []
        for s in seeds:
            e = run(dataset, [2,2,1], learning_rate=lr,target_error=0.1, seed=s, momentum=m, epochs=100, b=1)
            epochs.append(e)
        avg.append(np.mean(epochs))
        errors.append(np.std(epochs)/math.sqrt(len(epochs)))
    plt.errorbar(momentums, avg, yerr=errors, fmt='o-', label="lr = {}".format(lr))
plt.legend()
plt.xlabel("Momentum")
plt.ylabel("Epochs")
plt.xticks(momentums) 
plt.show()

In [ ]:

bs = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
learning_rates = [0.1, 0.5, 1]


plt.figure("avg epochs vs b")

for lr in learning_rates:
    avg = []
    errors = []

    for b in bs:
        epochs = []
        for s in seeds:
            e = run(dataset, [2,2,1], learning_rate=lr,target_error=0.1, seed=s, momentum=0, epochs=100, b=b)
            epochs.append(e)
        avg.append(np.mean(epochs))
        errors.append(np.std(epochs)/math.sqrt(len(epochs)))
    plt.errorbar(bs, avg, yerr=errors, fmt='o-', label="lr = {}".format(lr))
plt.legend()
plt.xlabel("b")
plt.xticks(bs)
plt.ylabel("Epochs") 
plt.show()

##### Single runs

In [ ]:
plt.figure("Error vs epoch")
rates = [0.01, 0.05, 0.1, 0.2,]
for rate in rates:
    network = Network(structure=([2, 2, 1]),
                      activation='tanh', seed=17, args={'b': 1})
    errors = network.train(dataset, learning_rate=rate,
                           epochs=1000, momentum=0)
    plt.plot(errors, label="Learning rate: {}".format(rate))
plt.legend()
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()


In [ ]:
plt.figure("Error vs epoch - momentum")
momentums = [0, 0.3, 0.7, 0.9]
for m in momentums:
    network = Network(structure=([2, 2, 1]),
                      activation='tanh', seed=17, args={'b': 1})
    errors = network.train(dataset, epochs=1000, momentum=m)
    plt.plot(errors, label="Momentum: {}".format(m))
plt.legend()
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()


In [ ]:
plt.figure("Error vs epoch - batch")
batch = [1,2,4]
for b in batch:
    network = Network(structure=([2, 2, 1]),
                      activation='tanh', seed=17, args={'b': 1})
    errors = network.train(dataset, epochs=1000, momentum=0, learning_rate=0.1, batch_size=b)
    plt.plot(errors, label="batch size: {}".format(b))
plt.legend()
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()


In [ ]:
plt.figure("Error vs epoch - b")
bs = [0.5,0.8,1,2,3,4,5,6,7,8]
for b in bs:
    network = Network(structure=([2, 2, 1]),
                      activation='tanh', seed=17, args={'b': b})
    errors = network.train(dataset, epochs=1000, momentum=0, learning_rate=0.001, batch_size=1)
    plt.plot(errors, label="b: {}".format(b))
plt.legend()
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()


##### Limit cases

In [ ]:
plt.figure("Error vs epoch - limits")
combinations = [ # lr bs m b label
    [0.9, 1, 0, 1, "learning rate: 0.9"],
    [0.1, 1, 1, 1, "momentum: 1"],
    [0.1, 1, 0, 4, "b: 4"]
]
for c in combinations:
    network = Network(structure=([2, 2, 1]),
                      activation='tanh', seed=17, args={'b': c[3]})
    errors = network.train(dataset, epochs=1000, momentum=c[2], learning_rate=c[0], batch_size=c[1])
    plt.plot(errors, label=c[4])
plt.legend()
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()


#### Animation

In [ ]:
imgs = []

size = 25
lim = 1
x = np.linspace(-lim, lim, size)
y = np.linspace(-lim, lim, size)

sample = [[i, j] for i in x for j in y]

plt.figure("temperature map animation")
plt.xlabel("E1")
plt.ylabel("E2")

mesh = plt.pcolormesh(x, y, np.zeros((size, size)), vmin=-1,
                      vmax=1, cmap="coolwarm", shading="gouraud")

plt.colorbar()

def callback(network):
    img = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            img[i][j] = network.feedforward([x[i], y[j]])[0]
    imgs.append(img)


def update(frame):
    mesh.set_array(np.array(imgs[frame]).ravel())
    return mesh


network = Network(structure=(
    [2, 2, 1]), activation='tanh', seed=17, args={'b': 1})
network.train(dataset, learning_rate=0.1, epochs=40,
              momentum=0.9, callback=callback)

# callback(network)
# update(len(imgs)-1)

# plt.show()

anim = animation.FuncAnimation(plt.gcf(), update, frames=len(imgs), interval=1)

anim.save('ej3.a.mp4', fps=30)

anim = None

update(len(imgs)-1)

plt.show()


#### More heatmaps

In [ ]:
lim = 2
size=200
x = np.linspace(-lim, lim, size)
y = np.linspace(-lim, lim, size)
sample = [[i, j] for i in x for j in y]

structures = [
    [2, 3, 1],
    [2, 15,10, 1],
    [2,10,15,1]
]


for s in structures:
    plt.figure("heatmap with {} neurons".format(s), figsize=(10, 5))
    network = Network(structure=s, activation='tanh', seed=17, args={'b': 0.8})
    errors = network.train(dataset, learning_rate=0.1,
                           target_error=0.01, momentum=0.9)
    print("Epochs: ", len(errors))
    print("Error: {:.2e}".format(network.error(dataset)))
    print("Structure: ", s)
    plot(network)
    plt.scatter([x[0] for x,y in dataset], [x[1] for x,y in dataset], c="black")
    plt.show()


### 3.b

In [ ]:
from datasets import number_images

dataset = []

for i, img in enumerate(number_images):
    dataset.append((np.array(img).flatten(), [1-i % 2]))


In [ ]:
network = Network(structure=([35, 20, 1]),
                  activation='sigmoid', seed=17, args={'b': 1})
errors = network.train(dataset, learning_rate=0.1, epochs=500, momentum=0.9)

print("Epochs: ", len(errors))
print("Error: {:.2e}".format(errors[-1]))

plt.figure("error - Par o impar")
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.plot(errors)
plt.show()

x = [i for i in range(10)]
y = [network.feedforward(dataset[i][0])[0] for i in x]

plt.figure("par o impar")
plt.scatter(x, y)
plt.xticks(x)
plt.ylabel("Impar - Par")
plt.xlabel("Numero")
plt.show()


In [ ]:
network_template = Network(structure=([35, 20, 1]),
                  activation='sigmoid', seed=17, args={'b': 1})        
lr = 0.1
m = 0.9
epsilon = 0.1
network,precision,precision_training,test,training = metrics.cross_validation(dataset, network_template, k=5,  epochs=100, metric = lambda n,d: metrics.accuracy(n,d,epsilon),lr=lr,m=m)
print("Precision test set: ",precision)
print("Precision for training set: ",precision_training)

test_acuracy = []
test_errors = []
training_acuracy = []
def callback(network):
    test_acuracy.append(metrics.accuracy(network,test,epsilon))
    test_errors.append(network.error(test))
    training_acuracy.append(metrics.accuracy(network,training,epsilon))

errors = network_template.train(training, epochs=100, learning_rate=lr, momentum=m, epoch_callback=callback)

plt.figure("Accuracy even odd")
plt.plot(test_acuracy, label="Test accuracy")
plt.plot(training_acuracy, label="Training accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.figure("Error even odd")
plt.plot(np.array(test_errors)/len(test), label="Test error")
plt.plot(np.array(errors)/len(training), label="Training error")
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.legend()
plt.show()

### 3.c

In [ ]:
from datasets import number_images

dataset = []

X = np.array([i for i in range(11)]) - 0.5

for i, img in enumerate(number_images):
    dataset.append((np.array(img).flatten(), [
                   1 if j == i else 0 for j in range(10)]))


In [ ]:
network = Network(structure=([35, 30, 20, 10]),
                  activation='sigmoid', seed=17, args={'b': 1})
errors = network.train(dataset, learning_rate=1, epochs=50, momentum=0)
print("Error: ", errors[-1])
print("Epochs: ", len(errors))
plt.figure("error - number recon")
plt.yscale('log')
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.plot(errors)
plt.show()

plt.figure("probability - number recon")

x = [i for i in range(10)]
matrix = []
for i in range(10):
    matrix.append(network.feedforward(dataset[i][0]))
    #plt.scatter(x,matrix[-1], label=i)
plt.pcolormesh(X, X, np.array(matrix).reshape(10, 10),
               cmap="coolwarm", vmin=0, vmax=1)
plt.colorbar()
plt.xlabel("Numero")
plt.ylabel("Certeza")
ticks = [i for i in range(10)]
plt.xticks(ticks)
plt.yticks(ticks)
plt.show()


#### Noise

In [ ]:
from utils import noise
noise_data = [[noise(img, 0.02, seed=17), output] for img, output in dataset]

plt.figure("probability - noise - number recon")

x = [i for i in range(10)]
matrix = []
for i in range(10):
    # lt.scatter(xxomatrix[-1], label=i
    matrix.append(network.feedforward(noise_data[i][0]))
print(network.error(noise_data))
plt.pcolormesh(X, X, np.array(matrix).reshape(10, 10),
               cmap="coolwarm", vmin=0, vmax=1)
plt.colorbar()
plt.xticks(x)
plt.yticks(x)
plt.xlabel("Numero")
plt.ylabel("Certeza")
plt.show()


In [ ]:
seeds = [i for i in range(1000)]

noise_data = []

for i in seeds:
    noise_data += [[noise(img, 0.02, seed=i), output]
                   for img, output in dataset]

classes = [output for img, output in dataset]
expected = [output for img, output in noise_data]
output = [network.feedforward(img) for img, output in noise_data]

mat, accuracy = metrics.confusion_matrix(classes, output, expected)

mat = mat.astype(int)

colours = []

for i in range(len(mat)-1):
    row = []
    for j in range(len(mat)-1):
        if i == j:
            row.append('#FFAAAA')
        else:
            row.append('#FFFFFF')
    row.append('#AAFFAA')
    colours.append(row)

row = []
for i in range(len(mat)):
    row.append('#AAFFAA')
colours.append(row)


labels = [f" {i} " for i in range(10)]
print("Accuracy: ", accuracy)
plt.figure("table", figsize=(10, 10))
plt.axis("off")
t = plt.table(cellText=mat,
          loc='center',
          cellLoc='center',
          rowLabels=labels + [" P"],
          colLabels=labels + ["R"],
          rowColours=['#FFFF00'] * 10 + ['#AAAAFF'],
          colColours=['#FFFF00'] * 10 + ['#AAAAFF'],
          cellColours=colours,
          )

t.set_fontsize(20)
t.scale(1,2)


plt.gcf().tight_layout()
plt.show()


### Optional

In [ ]:

def softmax(vector):
	e = np.exp(vector)
	return (e / e.sum(axis=1))*100
matrix = softmax(np.array([network.feedforward(IN, with_values=True)[1][-1] for IN, OUT in dataset]))

matrix =(100*matrix).astype(int)/100

colours = []

for i in range(len(matrix)):
    row = []
    for j in range(len(matrix)):
        if i == j:
            row.append('#FFAAAA')
        else:
            row.append('#FFFFFF')
    colours.append(row)

matrix  = [ [str(s)+"%" for s in row] for row in matrix]

plt.figure("softmax - number recon", figsize=(10, 10))
plt.axis("off")
labels = [f" {i} " for i in range(10)]

t = plt.table(cellText=matrix,
          loc='center',
          cellLoc='center',
            rowLabels=labels,
            colLabels=labels,
            rowColours=['#FFFF00'] * 10,
            colColours=['#FFFF00'] * 10,
            cellColours=colours,
          )

t.set_fontsize(20)
t.scale(1,2)


plt.gcf().tight_layout()
plt.show()